# Final (Fine Dining)

In [1329]:
import pandas as pd
import numpy as np
import datetime

In this final, you will be asked to analyze a data set containing historical reservations for a fine-dining restaurant that is stored in the file "data/Reservation_Data.csv".  This restaurant only accepts reservations booked online and hence does not accept walk in customers.   Each row in this data corresponds to a booked reservation.  The columns have the following meaning:
- reservation\_date: The date that the reservation was booked for. In other words, this is the date when the customer will dine.
- reservation\_time: The time that the reservation was booked for.
- reservation\_party\_size: The size of the party for the corresponding reservation, i.e. the number of diners.
- reservation_date_booked: the date on which the reservation was made.
- datetime\_booked: The date and time corresponding to when the reservation was made **(in UTC)**. This column has missing values, which have been entered as "#N/A".  **The restaurant is located in a Pacific time zone.**

Your goal in this final will be to understand how customers schedule reservations at this restaurant.


## Problem 1 

Your first task is to read in the data and do the following:

- delete rows with missing values
- convert datetime_booked to a datetime column with Pacific time zone (recall from above that these enteries are naive datetime, i.e., they do not have time zone information; they represent the date and time correspoinding to when the reservation was made in **UTC**)
- combine reservation_date and reservation time to create a new column called reservation_datetime that is a datetime column that has a Pacific time zone. (Recall that the restaurant is in Pacific time zone and these entries represent the time the customer will dine at the restaurant.) So the final dataframe you return should have 6 columns in total.
- Only keep reservations made at the following 8 time slots: 17:30, 17:45, 18:00, 18:15, 20:45, 21:00, 21:15, 21:30 (**Hint**: You need to use the reservation_datetime column to solve this).

Return this modified version of the original data frame.

In [1330]:
import pytz
def Read_Data():
    # Read in the data and drop rows with missing values
    df_res = pd.read_csv("data/Reservation_Data.csv", na_values=["#N/A"])
    df_res.dropna(inplace=True)
    # Convert datetime_booked to Pacific time zone
    df_res["datetime_booked"] = pd.to_datetime(df_res["datetime_booked"]).dt.tz_localize('UTC').dt.tz_convert('US/Pacific')
    # Combine reservation_date and reservation_time to create a new column called reservation_datetime in Pacific time zone
    df_res['reservation_datetime'] = pd.to_datetime(df_res['reservation_date'].str.cat(df_res['reservation_time'], sep=' '), format='%m/%d/%y %H:%M:%S').dt.tz_localize('US/Pacific')
    # Filter reservations by valid time slots
    time_slots = ['17:30', '17:45', '18:00', '18:15', '20:45', '21:00', '21:15', '21:30']
    df_res = df_res[df_res['reservation_datetime'].dt.strftime('%H:%M').isin(time_slots)]
    return df_res

In [1331]:
df_res = Read_Data()
assert df_res.shape==(3359,6)

In [1332]:
assert list(df_res.columns) == ['reservation_date', 'reservation_time', 'reservation_party_size',
                                'reservation_booked_date', 'datetime_booked', 'reservation_datetime']

In [1333]:
assert np.isclose(df_res.datetime_booked.dt.hour.mean(), 13.55969)

## Problem 2 

In this next part, we will write two functions to understand basic patterns in the data.

The first function takes as input one of the eight reservation time slots that we are considering as a datetime time and it outputs the day of the week (as a string) with the smallest average party size of all reservation made for the given inputted time slot.

In [1334]:
def Get_Avg_Party_Size(res_time):
    
    smallest_dow = None
    df_res = Read_Data()
    
    # Filter reservations by res_time
    df_res = df_res[df_res['reservation_datetime'].dt.strftime('%H:%M') == res_time.strftime('%H:%M')]
    
    # Group reservations by day of the week and calculate average party size
    avg_party_size_by_dow = df_res.groupby(df_res['reservation_datetime'].dt.dayofweek)['reservation_party_size'].mean()
    
    # Find day with smallest average party size
    smallest_avg_party_size = avg_party_size_by_dow.min()
    smallest_dow = avg_party_size_by_dow[avg_party_size_by_dow == smallest_avg_party_size].index[0]
    weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    smallest_dow = weekdays[smallest_dow]
    
    return smallest_dow

In [1335]:
res_time = datetime.time(hour = 17, minute=30)
assert Get_Avg_Party_Size(res_time)=="Sunday"

In [1336]:
res_time = datetime.time(hour = 21, minute=0)
assert Get_Avg_Party_Size(res_time)=="Tuesday"

## Problem 3
The second function I would like you to write takes as input a parameter called num_days (you may assume this is an integer), and it returns the fraction of reservations made by parties size 1/2 that are made at most num_days in advance, and then also the same fraction but just for parties of 3/4.

**NOTE: The number of days in advance that a reservation is made is (reservation_datetime - datetime_booked).**

In [1337]:
def Booked_in_Advance(num_days):
    
    df_res = Read_Data()
    num_days = datetime.timedelta(days=num_days)
    party_one_two = None
    party_three_four = None
    
    # Filter reservations for parties of size 1/2 and calculate fraction made <= num_days in advance
    df_res_1_2 = df_res[df_res['reservation_party_size'].isin([1,2])]
    total_num_res_1_2 = len(df_res_1_2)
    num_res_1_2_within_days = len(df_res_1_2[df_res_1_2['reservation_datetime'] - df_res_1_2['datetime_booked'] <= num_days])
    party_one_two = num_res_1_2_within_days / total_num_res_1_2
    
    # Filter reservations for parties of size 3/4 and calculate fraction made <= num_days in advance
    df_res_3_4 = df_res[df_res['reservation_party_size'].isin([3,4])]
    total_num_res_3_4 = len(df_res_3_4)
    num_res_3_4_within_days = len(df_res_3_4[df_res_3_4['reservation_datetime'] - df_res_3_4['datetime_booked'] <= num_days])
    party_three_four = num_res_3_4_within_days / total_num_res_3_4

    return party_one_two, party_three_four

In [1338]:
result = Booked_in_Advance(7)
assert np.isclose(result[0],0.27188)

In [1339]:
result = Booked_in_Advance(100)
assert np.isclose(result[1],0.860534)

## Problem 4 

Let's assume that there are two service periods:

- Service Period 1 consists of time slots 17:30, 17:45, 18:00, 18:15
- Service Period 2 consiss of time slots 20:45, 21:00, 21:15, 21:30

The purpose of this question is find out how popular a particular dining time is (within its own service period). The way we are going to go about this is to find out the "rank" of a dining time amoung all other bookings received on each day. To make the idea clear, consider dining time 17:45. If everyday the first booked reservation is for 17:45 (i.e., it has rank 1), then 17:45 is quite popular. On the other hand, if 17:45 is always the last dining time booked by customers on each day, it means that it is not that popular (perhaps it was booked because all other dining times were already booked by other customers). You are going to implement this idea in the next function.

The next function takes as input res_time, which is one of the eight times above as a datetime time.  Given the inputted time, you will exclusively focus this analysis on either Service Period 1 or 2. For example, if the inputted res_time is 17:45, you should only consider reservations made during Service Period 1. For each day, find the first reservation that books at res_time (datetime_booked tells you the order in which reservations were scheduled).  Within the given Service Period of interest, compute how many reservations have already been booked on that day before this reservation was made for res_time.  If $k-1$ reservations have already been made, this means that res_time was the $k$-th reservation that was booked on that day.  We will say the "rank" of reservation time res_time on that day was $k$. So, if res_time was the first time booked on a particular day for the relevant Service Period, its rank on that day is 1.  Return the average rank of res_time across all days in which a reservation for res_time was booked.

**Notice:** If there are two ranks which have the same booking time, they will have the same rank. For example, in the following table, both the first and second table share Rank 2. 

| reservation_dat | reservation_time | datetime_booked | rank |
|-----------------|------------------|-----------------|------|
|     9/9/17      |    18:15:00      |2017-08-15 10:03:00| 1|
|     9/9/17      |    18:00:00      |2017-08-15 10:04:00| 2|
|     9/9/17      |    17:45:00      |2017-08-15 10:04:00| 2|
|     9/9/17      |    17:30:00      |2017-08-15 10:05:00| 3|

In [1340]:
def Get_Avg_Rank(res_time):
    df_res = Read_Data()
    avg_Rank = None
    #Define service periods for the restaurant
    service_period_1 = ['17:30:00', '17:45:00', '18:00:00', '18:15:00']
    service_period_2= ['20:45:00', '21:00:00','21:15:00','21:30:00']
    #Combine service periods into one list 
    service_period = service_period_1 + service_period_2
    #Format the reservation time to be in the same format as the service periods
    res_time = res_time.strftime("%H:%M:%S")
    #Filter the data to include only the service period that the reservation time belongs to 
    for idx in service_period: 
        if res_time in service_period_1: 
            df_res=df_res.loc[df_res['reservation_time'].isin(service_period_1)]
        elif res_time in service_period_2:
            df_res=df_res.loc[df_res['reservation_time'].isin(service_period_2)]           
    groups_by_dt_booked = df_res.sort_values(by=["datetime_booked"])
    group_dt_booked_rdate=groups_by_dt_booked.groupby('reservation_date')
    num_reservations = 0
    sum_rank = 0
    for date,groups in group_dt_booked_rdate:
        #reset the index of the groups
        groups = groups.reset_index(drop=True)
        #filter groups to include only reservation time that matches input res_time
        matching_reservations = groups.loc[groups['reservation_time'] == res_time]
        if len(matching_reservations)>0:
        # Use the lowest index value as the rank
            rank = matching_reservations.index[0]  
        # Check if there are any other reservations with the same datetime_booked value
            same_dt_booked = groups[groups['datetime_booked'] == matching_reservations.iloc[0]['datetime_booked']]['reservation_time']
            if len(same_dt_booked) >= 2:
            # Assign the same rank to all reservation times with the same datetime_booked
                rank = same_dt_booked.index[0]   
            num_reservations += 1
            sum_rank += rank + 1
    avg_rank = sum_rank/num_reservations
    return avg_rank

In [1341]:
res_time = datetime.time(hour = 21, minute=30)
assert np.isclose(Get_Avg_Rank(res_time),3.9035)

In [1342]:
res_time = datetime.time(hour = 20, minute=45)
assert np.isclose(Get_Avg_Rank(res_time),1.45714)

In [1343]:
res_time = datetime.time(hour = 17, minute=30)
assert np.isclose(Get_Avg_Rank(res_time),3.17690)

In [1344]:
res_time = datetime.time(hour = 17, minute=45)
assert np.isclose(Get_Avg_Rank(res_time),2.77316)